In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd

import sklearn.model_selection as ms
import sklearn.preprocessing as p

import tensorflow as tf
import transformers as trfs
import os

In [ ]:
import tensorflow as tf
device = tf.test.gpu_device_name()
if device != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device))

Found GPU at: /device:GPU:0


In [ ]:
!nvidia-smi

Sat Sep 24 16:04:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    36W / 300W |   4709MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Max length of encoded string(including special tokens such as [CLS] and [SEP]):
MAX_SEQUENCE_LENGTH = 512

# BERT model with lowercase chars only:
PRETRAINED_MODEL_NAME = 'bert-base-uncased'

tokenizer = trfs.DistilBertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

def batch_encode(X, tokenizer):
    return tokenizer.batch_encode_plus(
    X,
    max_length=MAX_SEQUENCE_LENGTH, # set the length of the sequences
    add_special_tokens=True, # add [CLS] and [SEP] tokens
    return_attention_mask=True,
    return_token_type_ids=True, # not needed for this type of ML task
    pad_to_max_length=True,
    truncation = True, # add 0 pad tokens to the sequences less than max_length
    return_tensors='tf'
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


In [ ]:
new_model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/model/sum2/summodel")

# Check its architecture
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 bert (Custom>TFBertMainLayer)  {'pooler_output': (  109482240   ['input_ids[0][0]',              
                                None, 768),                       'attention_mask[0][0]',     

In [ ]:
cdir ="/content/drive/MyDrive/Colab Notebooks/data/testdata"

In [ ]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=b8c5146de7ccdc4279a0bfe9b901c3dad910f03e07ef71c1e8fe96fb21435eb2
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer
from statistics import mean
scorer = rouge_scorer.RougeScorer(['rouge1'])
results = {'precision': [], 'recall': [], 'fmeasure': []}
for subdir, dirs, files in os.walk(cdir):
  for file in files:
    f = os.path.join(subdir, file)
    print(f)
    df = pd.read_csv(f, delimiter=',', encoding= "utf-8")
    trueset=[]
    for index, row in df.iterrows():
      if (row["label2"] == 1):
        trueset.append(row["0"])
    print(trueset)
    xtest = df["0"].values
    ytest = df["label2"]
    xtest2 = batch_encode(xtest, tokenizer)
    y_pred = new_model.predict(xtest2.values(), batch_size=64, verbose=1)
    row_index=[]
    for i in enumerate(y_pred):
      if i[1][1] >= 0.50:
        row_index.append(i[0])
    #print(row_index)
    predsumm = []
    for j in enumerate(xtest):
      #print(j)
      if j[0] in row_index:
        predsumm.append(j[1])

    # for each of the hypothesis and reference documents pair
    for (h, r) in zip(trueset, predsumm):
        # computing the ROUGE
        score = scorer.score(h, r)
        # separating the measurements
        precision, recall, fmeasure = score['rouge1']
        # add them to the proper list in the dictionary
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['fmeasure'].append(fmeasure)

    print(results)
    for st,vals in results.items():
        print("Average for {} is {}".format(st,mean(vals)))

/content/drive/MyDrive/Colab Notebooks/data/testdata/Black-Box Variational Inference for Stochastic Differential Equations.csv
['A stochastic differential equation (SDE) defines a diffusion process, which evolves randomly over time, by describing its instantaneous behaviour.', 'As such, SDEs are powerful modelling tools used extensively in fields such as econometrics (Black & Scholes, 1973; Eraker, 2001), biology (Gillespie, 2000; Golightly & Wilkinson, 2011), physics (van Kampen, 2007) and epidemiology (Fuchs, 2013).\n', 'It is only possible to work with analytic solutions to SDEs in special cases.', 'Therefore it is common to use a numerical approximation, such as the Euler-Maruyama scheme.', 'Here the diffusion process is defined only on a grid of time points, and the transition density between successive diffusion states is approximated as Gaussian.', 'The RNN learns how to supply Gaussian state transitions between successive time points which closely match those for the intractabl

In [ ]:
from rouge_score import rouge_scorer
from statistics import mean
scorer = rouge_scorer.RougeScorer(['rougeL'])
results = {'precision': [], 'recall': [], 'fmeasure': []}
for subdir, dirs, files in os.walk(cdir):
  for file in files:
    f = os.path.join(subdir, file)
    print(f)
    df = pd.read_csv(f, delimiter=',', encoding= "utf-8")
    trueset=[]
    for index, row in df.iterrows():
      if (row["label2"] == 1):
        trueset.append(row["0"])
    print(trueset)
    xtest = df["0"].values
    ytest = df["label2"]
    xtest2 = batch_encode(xtest, tokenizer)
    y_pred = new_model.predict(xtest2.values(), batch_size=64, verbose=1)
    row_index=[]
    for i in enumerate(y_pred):
      if i[1][1] >= 0.50:
        row_index.append(i[0])
    #print(row_index)
    predsumm = []
    for j in enumerate(xtest):
      #print(j)
      if j[0] in row_index:
        predsumm.append(j[1])

    # for each of the hypothesis and reference documents pair
    for (h, r) in zip(trueset, predsumm):
        # computing the ROUGE
        score = scorer.score(h, r)
        # separating the measurements
        precision, recall, fmeasure = score['rouge1']
        # add them to the proper list in the dictionary
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['fmeasure'].append(fmeasure)

    print(results)
    for st,vals in results.items():
        print("Average for {} is {}".format(st,mean(vals)))

/content/drive/MyDrive/Colab Notebooks/data/testdata/Black-Box Variational Inference for Stochastic Differential Equations.csv
['A stochastic differential equation (SDE) defines a diffusion process, which evolves randomly over time, by describing its instantaneous behaviour.', 'As such, SDEs are powerful modelling tools used extensively in fields such as econometrics (Black & Scholes, 1973; Eraker, 2001), biology (Gillespie, 2000; Golightly & Wilkinson, 2011), physics (van Kampen, 2007) and epidemiology (Fuchs, 2013).\n', 'It is only possible to work with analytic solutions to SDEs in special cases.', 'Therefore it is common to use a numerical approximation, such as the Euler-Maruyama scheme.', 'Here the diffusion process is defined only on a grid of time points, and the transition density between successive diffusion states is approximated as Gaussian.', 'The RNN learns how to supply Gaussian state transitions between successive time points which closely match those for the intractabl

KeyError: ignored